In [28]:
#======================
#Sets libraries
#======================
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import json
import random
from datetime import datetime
import openpyxl

#======================
#Functions used
#======================
def set_wait(wait_type,time,selector_type,html_value):
    '''
    Description: Set waits
    '''
    by_selector = {
        'id': By.ID,
        'class': By.CLASS_NAME,
        'name': By.NAME,
        'tag': By.TAG_NAME,
        'link': By.LINK_TEXT,
        'partial_link': By.PARTIAL_LINK_TEXT,
        'css': By.CSS_SELECTOR,
        'xpath': By.XPATH
    }
    try:
        if wait_type=='for_presence':
            WebDriverWait(driver, time).until(
                EC.presence_of_element_located((by_selector[selector_type], html_value))
            )
        elif wait_type=='to_be_clickable':
                WebDriverWait(driver, time).until(
                EC.element_to_be_clickable((by_selector[selector_type],html_value))
            )        
    except TimeoutException:
        print("The program was ended becuase the time for the response was exceeded. Please try again")

def is_date_valid(date_str):
    '''
    Description: determines if an object is a valid date
    '''
    try:
        datetime.strptime(date_str, "%m/%d/%Y")
        return True
    except ValueError:
        return False

def get_header_index(arr_headers,header_text):
    '''
    Description: return the index of the header in a header array
    '''
    for i in range(len(arr_headers[0])):
        if arr_headers[0][i]==header_text:
            return i
        
def get_object_index(arr_objects,object_text):
    '''
    Description: return the index of the object in a object array
    '''
    for i in range(len(arr_objects)):
        if arr_objects[i].text==object_text:
            return i

def inject_data_to_excel(county,array_2d,excel_file_path):
    '''
    Description: function used to insert data from a scraped county into the webscraper file
    '''
    # Loads websraper workbook
    wb = openpyxl.load_workbook(filename=excel_file_path,read_only=False,keep_vba=True)
    # Creates/overwrites the county sheet
    if county not in wb.sheetnames:
        ws = wb.create_sheet(title=county)
        # Sets sheet headers
        arr_headers = ["No.", "SaleCode", "District Code", "County Name", "Parcel ID", "Parcel's URL", 
                       "Property Address", "Sec/Twp/Rng", "Recording", "Recording's URL", "Township", 
                       "Sale Date", "Seller Name", "Mailing Address", "Buyer Name", "Type of Sale", 
                       "Sale Price", "Assessed Land Value", "Assessed Building Value", "Assessed Dwelling Value", 
                       "Assessed Improvement Value", "Gross Assessed Value", "Exempt Value", "Net Assessed Value", 
                       "Detailed Land Subtotals", "Gross Acres", "Exempt Acres", "Net Acres", "CSR Points", 
                       "Avg. CSR", "Legal Description", "Document(s)"]
        ws.append(arr_headers)
    else:
        ws = wb[county]
        # Clears the sheet leaving the headers
        ws.delete_rows(2, ws.max_row - 1)
    
    # Agregar los datos desde la fila 2 en adelante
    # for row in array_2d:
    #     ws.append(row)
    
    # Guardar el archivo Excel
    wb.save(excel_file_path)

def get_random_user_agent():
    # Tu lista de User Agents en formato JSON
    user_agents_json = '[{"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.3", "pct": 34.12}, {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.3", "pct": 14.12}, {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.", "pct": 13.53}, {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.", "pct": 12.35}, {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.3", "pct": 4.71}, {"ua": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.4", "pct": 4.12}]'

    # Decodifica el JSON en un objeto Python
    user_agents = json.loads(user_agents_json)
    
    # Extrae la lista de User Agents
    ua_list = [ua_obj['ua'] for ua_obj in user_agents]
    
    # Escoge un User Agent al azar de la lista
    random_ua = random.choice(ua_list)
    
    return random_ua
  
#======================
#Asks for the county and time range
#======================
start_date='11/1/2023'
county='Buena Vista'
arr_counties = {
    'NW': ["BUENA VISTA 11", "CHEROKEE 18", "CLAY 21",
           "DICKINSON 30", "EMMET 32", "LYON 60", "OBRIEN 71",
           "OSCEOLA 72", "PALO ALTO 74", "PLYMOUTH 75",
           "POCAHONTAS 76", "SIOUX 84"],
    'NC': ["BUTLER 12", "CERRO GORDO 17", "FLOYD 34",
           "FRANKLIN 35", "HANCOCK 41", "HUMBOLDT 46", "KOSSUTH 55",
           "MITCHELL 66", "WINNEBAGO 95", "WORTH 98", "WRIGHT 99"],
    'NE': ["ALLAMAKEE 03", "BLACK HAWK 07", "BREMER 09",
           "BUCHANAN 10", "CHICKASAW 19", "CLAYTON 22", "DELAWARE 28",
           "DUBUQUE 31", "FAYETTE 33", "HOWARD 45", "WINNESHIEK 96"],
    'EC': ["BENTON 06", "CEDAR 16", "CLINTON 23", "IOWA 48",
           "JACKSON 49", "JOHNSON 52", "JONES 53", "LINN 57", "MUSCATINE 70",
           "SCOTT 82"],
    'C': ["BOONE 08", "DALLAS 25", "GRUNDY 38",
          "HAMILTON 40", "HARDIN 42", "JASPER 50", "MARSHALL 64",
          "POLK 77", "POWESHIEK 79", "STORY 85", "TAMA 86", "WEBSTER 94"],
    'SW': ["ADAIR 01", "ADAMS 02", "CASS 15", "FREMONT 36",
           "MILLS 65", "MONTGOMERY 69", "PAGE 73", "POTTAWATTAMIE 78",
           "TAYLOR 87"],
    'SC': ["APPANOOSE 04", "CLARKE 20", "DECATUR 27",
           "LUCAS 59", "MADISON 61", "MARION 63", "MONROE 68", "RINGGOLD 80",
           "UNION 88", "WARREN 91", "WAYNE 93"],
    'SE': ["DAVIS 26", "DES MOINES 29", "HENRY 44",
           "JEFFERSON 51", "KEOKUK 54", "LEE 56", "LOUISA 58", "MAHASKA 62",
           "VAN BUREN 89", "WAPELLO 90", "WASHINGTON 92"],
    'WC': ["AUDUBON 05", "CALHOUN 13", "CARROLL 14",
           "CRAWFORD 24", "GREENE 37", "GUTHRIE 39", "HARRISON 43", "IDA 47",
           "MONONA 67", "SAC 81", "SHELBY 83", "WOODBURY 97"]
}
ia_district=''
for district, counties in arr_counties.items():
    if county in counties:
        ia_district=district

# date_range=input('Please set the start and end date separated by a space using the mm/dd/yyyy format. If just one is set the end date will be today by default\n')
# if len(date_range.split())==1:
#     start_date=date_range.split()[0]
#     if not is_date_valid(start_date):
#         print('The start date is not valid. The program will end')
#         exit()
# else:
#     start_date=date_range.split()[0]
#     end_date=date_range.split()[1]
#     if not is_date_valid(start_date):
#         print('The start date is not valid. The program will end')
#         exit()
#     if not is_date_valid(end_date):
#         print('The end date is not valid. The program will end')
#         exit()

#======================
#Gets into the webpage
#======================
#Gets a fake user agent
user_agent=get_random_user_agent()
#Sets chr_options to pass the cloudflare captcha
chr_options = webdriver.ChromeOptions()
chr_options.add_experimental_option("detach", True)
chr_options.add_experimental_option("excludeSwitches", ["enable-automation"]) #Hides selenium browser features
chr_options.add_experimental_option('useAutomationExtension', False) #Hides selenium browser features
chr_options.add_argument("--disable-blink-features=AutomationControlled") #Hides testing browser features
chr_options.add_argument('--disable-extensions') #Disable extensions which may carry scripts
chr_options.add_argument('--no-sandbox') #Disable developer/sandbox mode
chr_options.add_argument('--disable-infobars') #Disable information bar
chr_options.add_argument('--disable-dev-shm-usage') #Disable memory optimization features
chr_options.add_argument('--disable-browser-side-navigation') #Disable memory optimization features
chr_options.add_argument('--disable-gpu') #Disable graphic features
chr_options.add_argument(f"user-agent={user_agent}")

#Sets the webdriver
chrdr_path="C:\Program Files (x86)\chromedriver.exe"
chrome_service=Service(executable_path=chrdr_path)
driver=webdriver.Chrome(service=chrome_service,options=chr_options)

#Opens 2 pages in the browser to simulate human behaviour
driver.get('https://www.google.com/')
# driver.switch_to.window(driver.window_handles[1])
# try:
#     # wait = WebDriverWait(driver, 5)
#     # no_thanks_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "button.M6CB1c.rr4y5c[aria-label='NO, GRACIAS']")))
#     no_thanks_button = WebDriverWait(driver, 15).until(
#         EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "NO, GRACIAS")]'))
#     )
#     # Hacer clic en el botón "NO, GRACIAS"
#     no_thanks_button.click()
# except:
#     print("No se encontró el botón 'NO, GRACIAS'")

search_box = driver.find_element(By.NAME,"q")
search_box.send_keys("Wikipedia")
search_box.send_keys(Keys.ENTER)
time.sleep(2)
first_result = driver.find_element(By.CSS_SELECTOR,"h3 a")
first_result.click()
time.sleep(2)
search_box = driver.find_element(By.CLASS_NAME,"cdx-text-input__input")[1]
search_box.send_keys("Lady Gaga")
search_box.send_keys(Keys.ENTER)

driver.execute_script("window.open('https://www.youtube.com/', '_blank')")
# driver.switch_to.window(driver.window_handles[1])
search_box = driver.find_element(By.NAME,"search_query")
search_box.send_keys("Lady Gaga")
search_box.send_keys(Keys.ENTER)
time.sleep(15)
driver.execute_script("window.open('https://beacon.schneidercorp.com/Application.aspx?AppID=1015&LayerID=21105&PageTypeID=2&PageID=9036', '_blank')")
# driver.switch_to.window(driver.window_handles[-1])

# driver.execute_script("window.open('https://www.google.com/','https://www.youtube.com/','https://beacon.schneidercorp.com/Application.aspx?AppID=1015&LayerID=21105&PageTypeID=2&PageID=9036')")
# driver.execute_script("window.open('https://beacon.schneidercorp.com/Application.aspx?AppID=1015&LayerID=21105&PageTypeID=2&PageID=9036','https://www.google.com/','https://www.youtube.com/')")
#Waits 15 seconds
# time.sleep(20)
#Moves to beacon schneider window
driver.switch_to.window(driver.window_handles[-1])
#Goes to the captcha frame
driver.switch_to.frame(0)
#Clicks on the chaptcha checkbox
driver.find_element(By.XPATH, '//*[@id="challenge-stage"]/div/label/input').click()
driver.switch_to.default_content()
#Goes to the captcha frame
# time.sleep(15)
# driver.switch_to.default_content()
# driver.switch_to.window(driver.window_handles[-1])
# driver.switch_to.frame(0)
# #Clicks on the chaptcha checkbox
# driver.find_element(By.XPATH, '//*[@id="challenge-stage"]/div/label/input').click()
#Goes back to default content
# driver.switch_to.default_content()

# #======================
# #Sets the parameters of the search
# #======================
# # Gets through terms and conditions
set_wait('for_presence',15,'xpath',"//a[text()='Agree']")
driver.find_element(By.XPATH,"//a[text()='Agree']").click()

# #Sets the sale price checkbox
# driver.find_element(By.ID,'ctlBodyPane_ctl02_ctl01_chkUseSalePrice').click()
# #Sets the sale price
# driver.find_element(By.ID,'ctlBodyPane_ctl02_ctl01_txtSalePriceLow').send_keys('1')

# #Sets sale date checkbox
# driver.find_element(By.ID,'ctlBodyPane_ctl02_ctl01_chkUseSaleDate').click()
# #Sets specific date range option button
# driver.find_element(By.ID,'ctlBodyPane_ctl02_ctl01_rdbUseSaleDateRange').click()
# #Sets start date
# driver.find_element(By.ID,'ctlBodyPane_ctl02_ctl01_txtSaleDateLow_VCS3Ag').send_keys(start_date)
# #Sets end date
# if 'end_date' in locals() or 'end_date' in globals():
#     driver.find_element(By.NAME,'ctlBodyPane$ctl02$ctl01$txtSaleDateHigh_VCS3Ag').send_keys(end_date)

# #Sets sale Nutc checkbox
# driver.find_element(By.ID,'ctlBodyPane_ctl02_ctl01_chkUseSaleNUTC').click()
# # Find the select element
# nutc_list = driver.find_element(By.ID,"ctlBodyPane_ctl02_ctl01_lstSaleNUTC")
# # Create a Select object
# select = Select(nutc_list)
# admisible_options = ['0 - Normal',
#     'Sale to/by Government/Exempt Organization',
#     'Transfer of partial interest',
#     'Life Estates',
#     'Adjoining or adjacent properties',
#     'Exchange, trade, gift, transfer from Estate',
#     'Change in Classification',
#     'Improvements or demolition after January 1 of the year of the sale',
#     'Court-ordered Sale',
#     'Auction sales',
#     'Split or Division',
#     'Sale of two or more parcels with different statutory classifications',
#     'Transfers to correct or modify conveyance',
#     'Sale of land without leased building',
#     'Other with explanation',
#     'STATE APPRAISALS',
#     'AGRICULTURAL SALE OF ONE OR MORE PARCELS',
#     'MULTIPLE PARCEL SALE',
#     "No consideration",
#     "Agricultural sale",
#     "Contract sale"
# ]
# for nutc_option in select.options:
#     nutc_option_text=nutc_option.text.lower()
#     for item in admisible_options:
#         if item.lower() in nutc_option_text:
#             nutc_option.click()

# #Does the search
# driver.find_element(By.ID,'ctlBodyPane_ctl02_ctl01_btnSearch').click()

# #======================
# #Works the results found
# #======================
# #Waits no matches notification
# WebDriverWait(driver,5)
# try:
#     message_markup=driver.find_element(By.ID,"ctlBodyPane_noDataList_pnlNoResults_sr")
#     print("No matches were found for this time range")
#     driver.quit()
# except NoSuchElementException:
#     pass

# #If the number of results is 200
# try:
#     total_results = driver.find_element(By.ID,'ctlBodyPane_ctl02_txtResultCount').text
#     if "Maximum" in total_results:
#         print("The time range is too large and it's leaving results out. Please try again reducing the time range")
#         driver.quit()  # Termina el programa si no hay resultados
# except NoSuchElementException:
#     pass

# #Gets the results table
# result_table=driver.find_element(By.ID,"ctlBodyPane_ctl02_ctl01_gvwAgCompResults")
# headers = result_table.find_elements(By.XPATH, ".//thead/tr")
# rows = result_table.find_elements(By.XPATH, ".//tbody/tr")
# arr_table= []
# arr_headers=[]
# # arr_headers=[['', 'Parcel ID', 'Address', 'Sale\nPrice', 'Sale\nDate', 'Sale NUTC', 'Assessed\nValue', 'Avg\nCSR', 'CSR\nPoints', 'Acres', 'DOV #', 'Multi\nParcel\nSale', 'Recording', '', '', 'parcel_id_url']]
# arr_matching_rows=[]
# #Loops over the table for headers
# for header in headers:
#     cells = header.find_elements(By.TAG_NAME, "th")
#     # Fills the rows with cells
#     header = [cell.text for cell in cells]
#     #Adds a blank header at the beginning
#     header.insert(0, "")
#     # Appends the row
#     arr_headers.append(header)
# #Adds header for parcel_id_url
# arr_headers[0].append('parcel_id_url')
# #Gets the column for the Multi\nParcel\nSale
# col_mps=get_header_index(arr_headers,'Multi\nParcel\nSale')
# #Gets the column for the Recording
# col_recording=get_header_index(arr_headers,'Recording')
# #Gets the Parcel ID column
# col_parcel_id=get_header_index(arr_headers,'Parcel ID')
# # Loops over the table for rows
# for row in rows:
#     cells = row.find_elements(By.TAG_NAME, "td")
#     # Fills the rows with cells
#     row = [cell.text for cell in cells]
#     # print(row)
#     parcel_id_url = cells[col_parcel_id].find_element(By.TAG_NAME, "a").get_attribute("href")
#     # print(parcel_id_url)
#     row.append(parcel_id_url)
#     # # Appends the row
#     arr_table.append(row)
# # Sort the table records based on the Recording
# arr_table=sorted(arr_table,key=lambda x: x[col_mps])
# # for row in arr_table:
# #     print(row)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="search-input"]"}
  (Session info: chrome=122.0.6261.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF725D64C82+3505170]
	(No symbol) [0x00007FF725990852]
	(No symbol) [0x00007FF725844145]
	(No symbol) [0x00007FF725889ADD]
	(No symbol) [0x00007FF725889C1C]
	(No symbol) [0x00007FF7258CAB27]
	(No symbol) [0x00007FF7258ABECF]
	(No symbol) [0x00007FF7258C83B2]
	(No symbol) [0x00007FF7258ABC33]
	(No symbol) [0x00007FF72587D618]
	(No symbol) [0x00007FF72587E6B1]
	GetHandleVerifier [0x00007FF725D967DD+3708781]
	GetHandleVerifier [0x00007FF725DEFC5D+4074477]
	GetHandleVerifier [0x00007FF725DE7DDF+4042095]
	GetHandleVerifier [0x00007FF725ABA136+708806]
	(No symbol) [0x00007FF72599CB0F]
	(No symbol) [0x00007FF725997D14]
	(No symbol) [0x00007

In [ ]:
def get_BV_record_data(record,get_arr_siblings=False):
    '''
    Description: get the data from record passed
    '''
    #'Parcel ID'
    parcel_id=record[1]
    #'Sale\nPrice'
    sale_price=record[3]
    #'Sale\nDate'
    sale_date=record[4]
    #'Assessed\nValue'
    assessed_value=record[6]
    #'Avg\nCSR'
    avg_csr=record[7]
    #'CSR\nPoints'
    csr_points=record[8]
    #'Acres'
    gross_acres=float(record[9])
    #'Recording'
    recording=record[12]
    #SaleCode
    sale_code=district+'//'+county.split(' ')[0].lower()+'//'+recording
    #'parcel_id_url']]
    parcel_id_url=record[15]
    #Set blanks variables
    assd_land_value='0'
    assd_building_value='0'
    assd_dwelling_value='0'
    assd_improvement_value='0'
    exempt_value='0'
    property_address=''
    sec_twp_rng=''
    legal_description=''
    documents=''
    township=''
    exempt_acres='0'
    net_acres='0'
    mailing_address=''
    seller=''
    buyer=''
    type_of_sale=''
    arr_siblings=[]
    #Goes into the record's webpage to get the rest of the data
    driver.execute_script("window.open('{}', '_blank');".format(parcel_id_url))
    driver.switch_to.window(driver.window_handles[-1])
    #Summary's Section - Skips if there's none
    if driver.find_element(By.ID,"ctlBodyPane_ctl00_mSection"):
        section_summary=driver.find_element(By.ID,"ctlBodyPane_ctl00_mSection")
        table_summary=section_summary.find_elements(By.CLASS_NAME,"tabular-data-two-column")[0]
        rows=table_summary.find_elements(By.TAG_NAME,"tr")
        #Loops over the rows on the summary's section
        exempt_acres=0
        for row in rows:
            #Proceeds only when th tags are found
            if row.find_elements(By.TAG_NAME,'th'):
                #Property Address
                if 'Property Address' in row.find_element(By.TAG_NAME,'th').text:
                    property_address=row.find_element(By.TAG_NAME,"td").text
                #Sec/Twp/Rng
                elif 'Sec/Twp/Rng' in row.find_element(By.TAG_NAME,"th").text:
                    sec_twp_rng=row.find_element(By.TAG_NAME,"td").text
                #Township
                elif 'Township' in row.find_element(By.TAG_NAME,"th").text:
                    township=row.find_element(By.TAG_NAME,"td").text                    
                #Brief Tax Description/Legal Description
                elif ('Brief Tax Description' in row.find_element(By.TAG_NAME,"th").text) or ('Legal Description' in row.find_element(By.TAG_NAME,"th").text):
                    legal_description=row.find_element(By.TAG_NAME,"td").text
                #Documents
                elif ('Documents' in row.find_element(By.TAG_NAME,"th").text):
                    documents=row.find_element(By.TAG_NAME,"td").text
                #Exempt Acres
                elif ('Exempt Acres' in row.find_element(By.TAG_NAME,"th").text):
                    exempt_acres=float(row.find_element(By.TAG_NAME,"td").text)

        #Net Acre
        if exempt_acres==0:
            net_acres=gross_acres
        else:
            net_acres=gross_acres-exempt_acres

    #Owners' Section - Skips if there's none
    if driver.find_element(By.ID,"ctlBodyPane_ctl02_mSection"):
        section_owners=driver.find_element(By.ID,"ctlBodyPane_ctl02_mSection")
        tbodies_owners=section_owners.find_elements(By.TAG_NAME,"tbody")
        #Mailing Address
        for tbody in tbodies_owners:
            rows=tbody.find_elements(By.TAG_NAME,'tr')
            if 'Mailing Address' in rows[0].text:
                mailing_address=rows[1].text

    #Sales' Section - Skips if there's none
    if driver.find_element(By.ID,"ctlBodyPane_ctl09_mSection"):
        section_sales=driver.find_element(By.ID,'ctlBodyPane_ctl09_mSection')
        table_sales=section_sales.find_element(By.TAG_NAME,'table')
        table_head=table_sales.find_element(By.TAG_NAME,'thead')
        table_headers=table_head.find_elements(By.TAG_NAME,'th')
        #Seller Name col
        seller_col=get_object_index(arr_objects=table_headers,object_text='Seller')
        #Buyer Name col
        buyer_col=get_object_index(arr_objects=table_headers,object_text='Buyer')
        #Type of sale col
        type_of_sale_col=get_object_index(arr_objects=table_headers,object_text='Type')
        #Sale data
        table_body=table_sales.find_element(By.TAG_NAME,'tbody')
        table_rows=table_body.find_elements(By.TAG_NAME,'tr')
        table_cells=table_rows[0].find_elements(By.XPATH,'./*')
        seller=table_cells[seller_col].text
        buyer=table_cells[buyer_col].text
        type_of_sale=table_cells[type_of_sale_col].text

    #Valuation's Section - Skips if there's none
    if driver.find_element(By.ID,"ctlBodyPane_ctl12_mSection"):
        section_valuation=driver.find_element(By.ID,'ctlBodyPane_ctl12_mSection')
        tbody_valuation=section_valuation.find_element(By.TAG_NAME,'tbody')
        rows_valuation=tbody_valuation.find_elements(By.TAG_NAME,'tr')
        for row in rows_valuation:
            #Assessed Land Value
            if 'Assessed Land Value' in row.find_element(By.TAG_NAME,'th').text:
                assd_land_value=row.find_elements(By.TAG_NAME,'td')[1].text
            #Assessed Building Value
            elif 'Assessed Building Value' in row.find_element(By.TAG_NAME,'th').text:
                assd_building_value=row.find_elements(By.TAG_NAME,'td')[1].text
            #Assessed Dwelling Value
            elif 'Assessed Dwelling Value' in row.find_element(By.TAG_NAME,'th').text:
                assd_dwelling_value=row.find_elements(By.TAG_NAME,'td')[1].text
            #Assessed Improvement Value
            elif 'Assessed Improvement Value' in row.find_element(By.TAG_NAME,'th').text:
                assd_improvement_value=row.find_elements(By.TAG_NAME,'td')[1].text
            #Exempt Value
            elif 'Exempt Value' in row.find_element(By.TAG_NAME,'th').text:
                exempt_value=row.find_elements(By.TAG_NAME,'td')[1].text

    #Gets an array of siblings records in case needed
    if get_arr_siblings:
        arr_siblings=[]

    driver.close()
    driver.switch_to.window(driver.window_handles[-1])

    return {
        'sale_code':sale_code,
        'parcel_id':parcel_id,  
        'sale_price':sale_price,
        'sale_date':sale_date,
        'assessed_value':assessed_value,
        'avg_csr':avg_csr,
        'csr_points':csr_points,
        'gross_acres':gross_acres,
        'recording':recording,
        'parcel_id_url':parcel_id_url,
        'assd_land_value':assd_land_value,
        'assd_building_value':assd_building_value,
        'assd_dwelling_value':assd_dwelling_value,
        'assd_improvement_value':assd_improvement_value,
        'exempt_value':exempt_value,
        'property_address':property_address,
        'sec_twp_rng':sec_twp_rng,
        'township':township,
        'legal_description':legal_description,
        'documents':documents,
        'exempt_acres':exempt_acres,
        'net_acres':net_acres,
        'mailing_address':mailing_address,
        'seller':seller,
        'buyer':buyer,
        'type_of_sale':type_of_sale,
        'arr_siblings':arr_siblings
    }

# Loops over all the records
arr_scraped_records=[]
for row in arr_table:
    #Proceeds if the record is multi parcel sale
    if row[col_mps]=='Y':
        #Gets the value to filter
        recording=row[col_recording]
        #Filter all the records with the same Recording
        arr_matching_rows.extend([row_matched for row_matched in arr_table if row_matched[6]==recording])
        #Proceeds if there is more than one record
        if len(arr_matching_rows)>1:
            #Loops over the filtered records 
            for record in arr_matching_rows:
                #Gets the data
                record_data=get_BV_record_data(record=record)
                arr_scraped_records.append(['', record_data['sale_code'], district, county, record_data['parcel_id'], '',
                            record_data['property_address'], record_data['sec_twp_rng'], record_data['parcel_id_url'],
                            record_data['township'], start_date, record_data['seller'], record_data['mailing_address'],
                            record_data['buyer'], record_data['type_of_sale'], record_data['sale_price'], record_data['assd_land_value'],
                            record_data['assd_building_value'], record_data['assd_dwelling_value'], record_data['assd_improvement_value'],
                            record_data['assessed_value'], record_data['exempt_value'], '', '', '', record_data['exempt_acres'],
                            record_data['net_acres'], record_data['csr_points'], record_data['avg_csr'], record_data['legal_description'],
                            record_data['documents']])
            #Proceeds if arr_scraped_records length is higher than 0
            if len(arr_scraped_records)>0:
                inject_data_to_excel(county=county.upper()+'-'+district,
                                     array_2d=arr_scraped_records,
                                     excel_file_path=r'C:\Users\USUARIO\Documents\Freelance\CLIENTES\DOLCE2014\WEBSCRAPER\WEBSCRAPER_v4.1 - copia.xlsm'
                                     )
        else:
            record_data=get_BV_record_data(record=record,get_arr_siblings=True)
        #Only one record - Means the record's page is not listing all the records and the info must be extracted within the visible record
            #Gets the data inside the record's webpage
            arr_scraped_records.append(['', record_data['sale_code'], district, county, record_data['parcel_id'], '',
                            record_data['property_address'], record_data['sec_twp_rng'], record_data['parcel_id_url'],
                            record_data['township'], start_date, record_data['seller'], record_data['mailing_address'],
                            record_data['buyer'], record_data['type_of_sale'], record_data['sale_price'], record_data['assd_land_value'],
                            record_data['assd_building_value'], record_data['assd_dwelling_value'], record_data['assd_improvement_value'],
                            record_data['assessed_value'], record_data['exempt_value'], '', '', '', record_data['exempt_acres'],
                            record_data['net_acres'], record_data['csr_points'], record_data['avg_csr'], record_data['legal_description'],
                            record_data['documents']])
            #Proceeds if arr_scraped_records length is higher than 0
            for item in record_data.arr_siblings:
                record_data=get_BV_record_data(record=item)
                if len(arr_scraped_records)>0:
                    inject_data_to_excel(county=county.upper()+'-'+district,
                                        array_2d=arr_scraped_records,
                                        excel_file_path=r'C:\Users\USUARIO\Documents\Freelance\CLIENTES\DOLCE2014\WEBSCRAPER\WEBSCRAPER_v4.1 - copia.xlsm'
                                        )
    else:
        #Not multi parcel sale
        #Gets the data
        record_data=get_BV_record_data(record=row)
        arr_scraped_records.append(['', record_data['sale_code'], district, county, record_data['parcel_id'], '',
                            record_data['property_address'], record_data['sec_twp_rng'], record_data['parcel_id_url'],
                            record_data['township'], start_date, record_data['seller'], record_data['mailing_address'],
                            record_data['buyer'], record_data['type_of_sale'], record_data['sale_price'], record_data['assd_land_value'],
                            record_data['assd_building_value'], record_data['assd_dwelling_value'], record_data['assd_improvement_value'],
                            record_data['assessed_value'], record_data['exempt_value'], '', '', '', record_data['exempt_acres'],
                            record_data['net_acres'], record_data['csr_points'], record_data['avg_csr'], record_data['legal_description'],
                            record_data['documents']])
        #Proceeds if arr_scraped_records length is higher than 0
        if len(arr_scraped_records)>0:
            inject_data_to_excel(county=county.upper()+'-'+district,
                                array_2d=arr_scraped_records,
                                excel_file_path=r'C:\Users\USUARIO\Documents\Freelance\CLIENTES\DOLCE2014\WEBSCRAPER\WEBSCRAPER_v4.1 - copia.xlsm'
                                )

KeyboardInterrupt: 